In [1]:
import string
from collections import defaultdict
import numpy as pd
import numpy as np
from argparse import Namespace

In [2]:
config = Namespace(
    file_path = './data/shakespeare.txt'
)

## Load data

In [3]:
vocab  = defaultdict(int)

with open(config.file_path) as f:
    for line in f.readlines():
        tokens = line.strip().split(" ")
        for token in tokens:
            vocab[token.lower()] += 1
            
vocab = {k:v for k,v in vocab.items() if v>1 and len(k)>1}
print(f"Number of words in vocab : {len(vocab)}")
vocab = {k: np.round(v/len(vocab),3) for k,v in vocab.items()}

Number of words in vocab : 3547


## Auto- Correct Algorithm

    1. Identify the misspelled word (word not in vocab)
    2. Use delete, insert, replace, switch operation to get all words
    3. Filter words that are in vocab
    4. Highest prbability word will be more likely right word.

## Utilities

In [4]:
def split_word(w):
    return [(w[:i], w[i:]) for i in range(len(w))]

def delete_letter(word):
    split_words = split_word(word)
    out_words = [word_l[:-1]+word_r for word_l, word_r in split_words if len(word_l)>0]
    return out_words

def switch_letter(word):
    split_words = split_word(word)
    out_words = [word_l[:-1]+word_r[0]+word_l[-1:]+word_r[1:] for word_l, word_r in split_words if len(word_l)>0]
    return out_words

def insert_letter(word):
    split_words = split_word(word)
    letters = list(string.ascii_lowercase)
    out_words = set()
    
    for word_l, word_r in split_words:
        for letter in letters:
            final_word = word_l + letter + word_r
            out_words.add(final_word)
    
    out_words = list(out_words - {word})
    
    return out_words

def replace_letter(word):
    out_words = []
    
    delete_words = delete_letter(word)
    for w in delete_words:
        out_words += insert_letter(w)
    out_words = list(set(out_words) - {word})
    return out_words

def one_edit_distance_words(word):
    
    words = []
    words += delete_letter(word)
    words += insert_letter(word)
    words += replace_letter(word)
    words += switch_letter(word)
    words = list(set(words))
    
    return words

def two_edit_distance_words(word):
    
    one_edit_dist_words = one_edit_distance_words(word)
    
    two_edit_dist_words = []
    for w in one_edit_dist_words:
        two_edit_dist_words += one_edit_distance_words(w)
    
    two_edit_dist_words = list(set(two_edit_dist_words) - {word})
    
    return two_edit_dist_words

In [5]:
## Find the min edit distance between two string using dynamic programming
def min_edit_distance(source, target, ins_cost = 1, del_cost = 1, rep_cost = 2):
    
    m = len(source)
    n = len(target)

    D = np.zeros((m+1, n+1), dtype=int) 
    
    for row in range(1,m+1):
        D[row,0] = D[row-1,0] + del_cost
        
    for col in range(1,n+1): 
        D[0,col] = D[0,col-1] + ins_cost
        
    for row in range(1,m+1): 
        for col in range(1,n+1):
            
            r_cost = rep_cost
            if source[row-1] == target[col-1]:
                r_cost = 0
            D[row,col] = min([D[row-1,col]+del_cost, D[row,col-1]+ins_cost, D[row-1,col-1]+r_cost])
          
    med = D[m,n]
    
    return D, med

In [6]:
def auto_correct(word, vocab):
    
    if word in vocab:
        return word
    else:
        one_edit_words = one_edit_distance_words(word)
        one_freqs_dist = {w:vocab[w] for w in one_edit_words if w in vocab}
        
        two_edit_words = two_edit_distance_words(word)
        two_freqs_dist = {w:vocab[w] for w in two_edit_words if w in vocab}
        
        if one_freqs_dist is not None:
            one_freqs_words = sorted(one_freqs_dist, key = lambda x: one_freqs_dist[x], reverse = True)
            
        if two_freqs_dist is not None:
            two_freqs_words = sorted(two_freqs_dist, key = lambda x: two_freqs_dist[x], reverse = True) ## decending order
    
    final_words = list(set(one_freqs_words + two_freqs_words[:5]))
    
    
    ## Use min edit cost to suggest the words
    costs = {}
    for w in final_words:
        costs[w] = min_edit_distance(word, w)[1]
        
    if costs != {}:
        suggestions = sorted(costs, key = lambda x: costs[x])  ## ascending
        
    return suggestions

In [7]:
word = "you"
print(f"Original Word : {word} \t Correct Words: {auto_correct(word,vocab)}")

word = "yuo"
print(f"Original Word : {word} \t Correct Words: {auto_correct(word,vocab)}")

word = "heanve"
print(f"Original Word : {word} \t Correct Words: {auto_correct(word,vocab)}")

word = "sholud"
print(f"Original Word : {word} \t Correct Words: {auto_correct(word,vocab)}")

Original Word : you 	 Correct Words: you
Original Word : yuo 	 Correct Words: ['you', 'to', 'so', 'do', 'no', 'who']
Original Word : heanve 	 Correct Words: ['heaven', 'have', 'leave', 'change', 'whence']
Original Word : sholud 	 Correct Words: ['sold', 'hold', 'should', 'sound', 'behold']
